In [1]:
#LOAD PACKAGES
#-------------
using JuMP  # Need to say it whenever we use JuMP
using GLPKMathProgInterface # Loading the GLPK module for using its solver

#MODEL CONSTRUCTION
#------------------
sfMipModel = Model(solver = GLPKSolverMIP()) # This time, we use the MIP solver! 

┌ Info: Precompiling GLPKMathProgInterface [3c7084bd-78ad-589a-b5bb-dbd673274bea]
└ @ Base loading.jl:1192
ERROR: LoadError: LoadError: InitError: UndefVarError: libglpk not defined
Stacktrace:
 [1] version at C:\Users\Striker\.julia\packages\GLPK\bSnuc\src\GLPK.jl:227 [inlined]
 [2] __init__() at C:\Users\Striker\.julia\packages\GLPK\bSnuc\src\GLPK.jl:247
 [3] _include_from_serialized(::String, ::Array{Any,1}) at .\loading.jl:633
 [4] macro expansion at .\logging.jl:310 [inlined]
 [5] _require_search_from_serialized(::Base.PkgId, ::String) at .\loading.jl:704
 [6] _require(::Base.PkgId) at .\loading.jl:937
 [7] require(::Base.PkgId) at .\loading.jl:858
 [8] macro expansion at .\logging.jl:309 [inlined]
 [9] require(::Module, ::Symbol) at .\loading.jl:840
 [10] include at .\boot.jl:317 [inlined]
 [11] include_relative(::Module, ::String) at .\loading.jl:1044
 [12] include at .\sysimg.jl:29 [inlined]
 [13] include(::String) at C:\Users\Striker\.julia\packages\GLPKMathProgInterface\8q6jt

ErrorException: Failed to precompile GLPKMathProgInterface [3c7084bd-78ad-589a-b5bb-dbd673274bea] to C:\Users\Striker\.julia\compiled\v1.0\GLPKMathProgInterface\Y5bTM.ji.

In [15]:
#INPUT DATA
#----------

n = 5 # no. of continuous decision variables
p = 4 # no. of integer decision variables
m = 3 # no. of constraints

# coefficients
A = [0.7511 -0.1357 0.7955 -0.4567 0.1356; -0.6670 -0.3326 0.1657 -0.5519 -0.9367; 1.5894 -0.1302 -0.4313 -0.4875 0.4179]
B = [-0.09520 -0.28056 -1.33978 0.6506; -0.8581 -0.3518 1.2788 1.5114; -0.5925 1.3477 0.1589 0.03495]
c = [0.3468 0.8687 0.1200 0.5024 0.2884]
d = [0.2017 0.2712 0.4997 0.9238]
f = [0.1716 0.3610 0.0705]

1×3 Array{Float64,2}:
 0.1716  0.361  0.0705

In [6]:
#VARIABLES
#---------

# We work with the two vectors of decision variables
@variable(sfMipModel, x[1:n] >=0) # continuous
@variable(sfMipModel, y[1:p] >= 0, Int) # integer 

4-element Array{Variable,1}:
 y[1]
 y[2]
 y[3]
 y[4]

In [11]:
#CONSTRAINTS
#-----------

for i = 1:m
    @constraint(sfMipModel, sum{A[i,j]*x[j], j=1:n} + sum{B[i,j]*y[j], j=1:p} == f[i]) 
end

In [12]:
#OBJECTIVE
#---------

# Minimize cTx
@objective(sfMipModel, Min, sum{c[i] * x[i], i in 1:n} + sum{d[i]*y[i], i in 1:p}) 


#PRINT THE MODEL IN A HUMAN-READABLE FORMAT
#------------------------------------------
println("The optimization problem to be solved is:")
print(sfMipModel) # Shows the model constructed in a human-readable form

The optimization problem to be solved is:
Min 0.3468 x[1] + 0.8687 x[2] + 0.12 x[3] + 0.5024 x[4] + 0.2884 x[5] + 0.2017 y[1] + 0.2712 y[2] + 0.4997 y[3] + 0.9238 y[4]
Subject to
 0.7511 x[1] - 0.1357 x[2] + 0.7955 x[3] - 0.4567 x[4] + 0.1356 x[5] - 0.0952 y[1] - 0.28056 y[2] - 1.33978 y[3] + 0.6506 y[4] = 0.1716
 -0.667 x[1] - 0.3326 x[2] + 0.1657 x[3] - 0.5519 x[4] - 0.9367 x[5] - 0.8581 y[1] - 0.3518 y[2] + 1.2788 y[3] + 1.5114 y[4] = 0.361
 1.5894 x[1] - 0.1302 x[2] - 0.4313 x[3] - 0.4875 x[4] + 0.4179 x[5] - 0.5925 y[1] + 1.3477 y[2] + 0.1589 y[3] + 0.03495 y[4] = 0.0705
 x[i] ≥ 0 ∀ i ∈ {1,2,3,4,5}
 y[i] ≥ 0, integer, ∀ i ∈ {1,2,3,4}


In [13]:
#SOLVE IT
#--------

status = solve(sfMipModel) # solves the model  

:Optimal

In [14]:
#DISPLAY THE RESULTS
#-------------------

println("Objective value: ", getobjectivevalue(sfMipModel)) 
println("x = ", getvalue(x))
println("y = ", getvalue(y))

Objective value: 1.070277955983598
x = [0.0654907, 0.0, 1.62986, 0.0, 1.22151]
y = [0.0, 0.0, 1.0, 0.0]
